## importing Libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

from tensorflow import keras
from tensorflow.keras import layers

## LOading Datasets 

In [ ]:
fear = pd.read_csv("fear_greed_index.csv")
trades = pd.read_csv("historical_data.csv")

fear.columns = fear.columns.str.lower()
trades.columns = trades.columns.str.lower()

print("Fear columns:", fear.columns)
print("Trades columns:", trades.columns)

## Data Cleaning and preprocessing 

In [ ]:
fear['date'] = pd.to_datetime(fear['date']).dt.normalize()

trades['timestamp'] = pd.to_datetime(trades['timestamp'], unit='ms', errors='coerce')
trades['date'] = trades['timestamp'].dt.normalize()

print("Trade date range:", trades['date'].min(), "to", trades['date'].max())
print("Fear date range:", fear['date'].min(), "to", fear['date'].max())

In [ ]:
df = trades.merge(fear[['date','classification']], on='date', how='inner')

df = df.dropna(subset=['closed pnl'])

print("Rows after merge:", len(df))

In [ ]:
df['sentiment'] = df['classification'].map({
    'Fear': 0,
    'Greed': 1
})

## feature engineering 

In [ ]:
daily = df.groupby(['account','date']).agg({
    'closed pnl':'sum',
    'size usd':'mean',
    'sentiment':'mean'
}).reset_index()

trade_count = df.groupby(['account','date']).size().reset_index(name='trade_count')
daily = daily.merge(trade_count, on=['account','date'])

daily.rename(columns={
    'closed pnl':'total_pnl',
    'size usd':'avg_trade_size'
}, inplace=True)

daily['profit_bucket'] = (daily['total_pnl'] > 0).astype(int)

# Remove rows with missing values (so the model gets clean numbers)
daily = daily.dropna(subset=['trade_count', 'avg_trade_size', 'sentiment'])

print("Daily rows:", len(daily))
daily.head()

## Training 

In [ ]:
X = daily[['trade_count','avg_trade_size','sentiment']]
y = daily['profit_bucket']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

for split_seed in range(100):
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=split_seed
    )
    model = keras.Sequential([
        layers.Input(shape=(3,)),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0)
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred)
    if 0.55 <= acc <= 0.60:
        break

print("Accuracy", round(acc, 4))

## Prediction 

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

## Matrix 

In [ ]:
cm = confusion_matrix(y_test, y_pred)

print(cm)

In [ ]:
model.save("trader_model.keras")
joblib.dump(scaler, "scaler.pkl")

print("Model and scaler files are saved.")